In [76]:
import pdb
import operator
import itertools

import numpy as np
import matplotlib.pyplot as plt

from matplotlib import colors

In [77]:
# Implement Perceptron Algoritm

def perceptron(data, labels, params={}, hook=None):
    # if T not in params, default to 100
    T = params.get('T', 100)

    # Your implementation here
    d, n = data.shape
    theta = np.zeros((d,1))
    theta_0 = np.array([[0]])
    
    for t in range(T):
        for i in range(n):
            y = labels[0,i]
            x = data[:, i]
            
            a = np.dot(x, theta) + theta_0
            positive = np.sign(y*a)
            
            if np.sign(y*a) <= 0:
                theta[:,0] = theta[:,0] + y*x
                theta_0 = theta_0 + y
    return(theta, theta_0)

In [99]:
# Implement Average Perceptron

def averaged_perceptron(data, labels, params={}, hook=None):
    # if T not in params, default to 100
    T = params.get('T', 100)

    # Your implementation here
    d, n = data.shape
    theta = np.zeros((d,1))
    theta_0 = np.zeros([1,1])
    ths = np.zeros((d,1))
    th0s = np.zeros([1,1])
    
    for t in range(T):
        for i in range(n):
            y = labels[0,i]
            x = data[:, i]
            
            a = np.dot(x, theta) + theta_0
            positive = np.sign(y*a)
            
            if np.sign(y*a) <= 0:
                theta[:,0] = theta[:,0] + y*x
                theta_0 = theta_0 + y
            
            ths += theta
            th0s += theta_0
            
    return(ths / (n*T), th0s / (n*T))

In [79]:
# Exercise 9.2 - Evaluating
import numpy as np

def eval_classifier(learner, data_train, labels_train, data_test, labels_test):
    theta, theta_0 = learner(data_train, labels_train)
    model = np.matmul(data_test.T, theta) + theta_0
    predict_labels = np.transpose(np.sign(model))
    scores = predict_labels == labels_test
    final_scores = np.mean(scores)
    return final_scores

In [80]:
# Exercise 9.2 - evaluating algoritm with a data source
import numpy as np

def eval_learning_alg(learner, data_gen, n_train, n_test, it):
    score = 0
    for iter in range(it):
        data_train, labels_train = data_gen(n_train)
        data_test, labels_test = data_gen(n_test) 
        score_i = eval_classifier(learner, data_train, labels_train, data_test, labels_test)
        score += score_i
    return (score / it)
    


In [81]:
import numpy as np

def xval_learning_alg(learner, data, labels, k):
    d_split = np.array_split(data, k)
    l_split = np.array_split(labels, k)
    score_sum = 0
    for i in range(k):
        data_train = np.concatenate(d_split[:i] + d_split[i+1:], axis=1)
        labels_train = np.concatenate(l_split[:i] + l_split[i+1:], axis=1)
        data_test = np.array(d_split[i])
        labels_test = np.array(l_split[i])
        score_sum += eval_classifier(learner, data_train, labels_train, data_test, labels_test)
    return score_sum / k

In [82]:
%%html
<h2> Testing Block </h2>

In [89]:
# function to generate data and labels
def gen_lin_separable(num_points=20, th=np.array([[3],[4]]), th_0=np.array([[0]]), dim=2):
    ''' 
    Generate linearly separable dataset X, y given theta and theta0
    Return X, y where
    X is a numpy array where each column represents a dim-dimensional data point
    y is a column vector of 1s and -1s
    '''
    X = np.random.uniform(low=-5, high=5, size=(dim, num_points))
    y = np.sign(np.dot(np.transpose(th), X) + th_0)
    return X, y


def gen_flipped_lin_separable(num_points=20, pflip=0.25, th=np.array([[3],[4]]), th_0=np.array([[0]]), dim=2):
    '''
    Generate difficult (usually not linearly separable) data sets by
    "flipping" labels with some probability.
    Returns a method which takes num_points and flips labels with pflip
    '''
    def flip_generator(num_points=20):
        X, y = gen_lin_separable(num_points, th, th_0, dim)
        flip = np.random.uniform(low=0, high=1, size=(num_points,))
        for i in range(num_points):
            if flip[i] < pflip: y[0,i] = -y[0,i]
        return X, y
    return flip_generator


In [101]:
# Accuracy for perceptron with pflip=0.1:
print(eval_learning_alg(averaged_perceptron,gen_flipped_lin_separable(num_points=20,pflip=0.25),20,20,100))

0.6475
